In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.feature_extraction.text import TfidfVectorizer
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('survey responses.csv')
print('This dataset include {} rows and {} columns'.format(*df.shape))

This dataset include 343 rows and 23 columns


In [3]:
df

,Current program,Year of study,Program level,Sex,Ethnicity,Age group,Preferred learning style,Used ChatGPT for learning purposes,How often ChatGPT is used for educational\npurposes,ChatGPT usage purposes,...,Improvement of marks\nsince using ChatGPT for personalized learning,Level of confidence in understanding of\nsubjects after using ChatGPT,Sample situations where\nChatGPT significantly aided learning,Used any features of ChatGPT that tailor\nresponses to learning preferences or\nstyles,"If yes, how effective are the tailored\nresponses in enhancing comprehension and\nknowledge acquisition",Feel that personalized responses from\nChatGPT have improved grades or academic\nperformance,Improvements to suggest for ChatGPT\nto better support learning,Think that integrating ChatGPT more deeply into\nthe curriculum would benefit students,Features that would like to be seen in ChatGPT to\nenhance personalized learning capabilities,Additional comments or suggestions regarding\nthe use of ChatGPT in learning process
0,CSIS,Second,Postgraduate,Male,Hispanic/Latino,25-34,Auditory,Yes,Weekly,"Understanding concepts, Exam Preparation",...,No change,Neutral,NaN,No,NaN,Neutral,NaN,Disagree,NaN,NaN
1,CSIS,First,Diploma,Female,Asian,25-34,Auditory,Yes,Daily,"Understanding concepts, Assignments, Tutoring ...",...,"Yes, significant improvement",Very confident,Ask for explaining some concept,Yes,Very effective,Agree,"The way ChatGPT rectify answers that I asked, ...",Strongly Agree,Visualization,NaN
2,CSIS,Second,Postgraduate,Female,Asian,25-34,Auditory,No,Never,Understanding concepts,...,No change,Neutral,Explain difficult concepts,No,Neutral,Neutral,NaN,Neutral,More examples,Not use for exams
3,Business,Third,Bachelors,Male,Asian,25-34,Auditory,No,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CSIS,Second,Postgraduate,Male,Asian,25-34,Auditory,Yes,Daily,"Understanding concepts, Homework assistance, A...",...,"Yes, slight improvement",Confident,When I search for concepts that I don't unders...,Yes,Very effective,Agree,more accurate amd don't fake citations,Strongly Agree,Summarize lectures,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,Business,Second,Bachelors,Female,Asian,35-44,"Visual, Reading/Writing",Yes,Rarely,Project research,...,Decrease in grades,Confident,Assisted in learning a new programming languag...,Yes,Effective,Neutral,Enhancing the feedback system for educational ...,Strongly Agree,NaN,NaN
339,Business,Third,Diploma,Prefer not to say,Asian,18-24,"Visual, Reading/Writing",Yes,Monthly,Exam Preparation,...,Decrease in grades,Not applicable,Provided additional practice problems for math,Yes,Very effective,Disagree,Enhancing the summarization capabilities for e...,Neutral,NaN,NaN
340,Business,Third,Postgraduate,Female,Hispanic/Latino,Prefer not to say,"Visual, Reading/Writing, Usage Patterns",Yes,Daily,"Understanding concepts, Assignments, Class lec...",...,"Yes, significant improvement",Very confident,NaN,No,NaN,Strongly Agree,Enhancing the feedback system for educational ...,Strongly Agree,adaptive learning algorithms,I would recommend incorporating ChatGPT into e...
341,Business,First,Diploma,Female,Pacific Islander,25-34,"Visual, Reading/Writing, Usage Patterns",Yes,Rarely,Homework assistance,...,No change,Very confident,NaN,No,NaN,Strongly disagree,Introducing personalized study plans,Neutral,peer collaboration features,I have had a positive experience using ChatGPT...


In [4]:
print(df.isnull().sum())

Current program                                                                                              0
Year of study                                                                                                0
Program level                                                                                                0
Sex                                                                                                          0
Ethnicity                                                                                                    0
Age group                                                                                                    0
Preferred learning style                                                                                     0
Used ChatGPT for learning purposes                                                                           0
How often  ChatGPT is used for educational\npurposes                                                        28
C

In [5]:
# Get the indices of the last two rows
last_two_indices = df.index[-2:]

# Drop the last two rows
df = df.drop(last_two_indices)

In [6]:
# Custom Value Imputation
custom_values = {
    'How often  ChatGPT is used for educational\npurposes': 'Not Applicable',
    'ChatGPT usage purposes': 'Not Applicable',
    'ChatGPT usage hours per week': 'Not Applicable',
    'Satisfaction with the quality of responses by ChatGPT': 'Not Applicable',
    'How helpful ChatGPT in improving the\nunderstanding of course material': 'Not Applicable',
    'Improvement of marks\nsince using ChatGPT for personalized learning': 'Not Applicable',
    'Level of confidence in understanding of\nsubjects after using ChatGPT': 'Not Applicable',
    'Sample situations where\nChatGPT significantly aided learning': 'No comment',
    'Used any features of ChatGPT that tailor\nresponses to  learning preferences or\nstyles': 'Not Applicable',
    'If yes, how effective are the tailored\nresponses in enhancing comprehension and\nknowledge acquisition': 'Not Applicable',
    'Feel that personalized responses from\nChatGPT have improved grades or academic\nperformance': 'Not Applicable',
    'Improvements to suggest for ChatGPT\nto better support learning': 'No comment',
    'Think that integrating ChatGPT more deeply into\nthe curriculum would benefit students': 'Not Applicable',
    'Features that would like to be seen in ChatGPT to\nenhance  personalized learning capabilities': 'No comment',
    'Additional comments or suggestions regarding\nthe use of ChatGPT in learning process': 'No comment'
}

# Apply custom value imputation
df = df.fillna(value=custom_values)

In [7]:
print(df.isnull().sum())

Current program                                                                                            0
Year of study                                                                                              0
Program level                                                                                              0
Sex                                                                                                        0
Ethnicity                                                                                                  0
Age group                                                                                                  0
Preferred learning style                                                                                   0
Used ChatGPT for learning purposes                                                                         0
How often  ChatGPT is used for educational\npurposes                                                       0
ChatGPT usage purpo

In [8]:
# Convert 'Preferred learning style' to categorical values
learning_style_dummies = df['Preferred learning style'].str.get_dummies(sep=', ')

# Convert 'ChatGPT usage purposes' to categorical values
usage_purposes_dummies = df['ChatGPT usage purposes'].str.get_dummies(sep=', ')

# Concatenate the new dummy columns to the original DataFrame
df = pd.concat([df, learning_style_dummies, usage_purposes_dummies], axis=1)

# Drop the original columns if no longer needed
df.drop(['Preferred learning style', 'ChatGPT usage purposes'], axis=1, inplace=True)

print("\nDataFrame with Categorical Values:")
print(df)



DataFrame with Categorical Values:
    Current program Year of study Program level                Sex  \
0              CSIS        Second  Postgraduate               Male   
1              CSIS         First       Diploma             Female   
2              CSIS        Second  Postgraduate             Female   
3          Business         Third     Bachelors               Male   
4              CSIS        Second  Postgraduate               Male   
..              ...           ...           ...                ...   
336        Business         First       Diploma  Prefer not to say   
337        Business        Second   Certificate               Male   
338        Business        Second     Bachelors             Female   
339        Business         Third       Diploma  Prefer not to say   
340        Business         Third  Postgraduate             Female   

             Ethnicity          Age group Used ChatGPT for learning purposes  \
0      Hispanic/Latino              25-34  

In [9]:
# Define the categorical columns and possible categories
categorical_columns = {
    'Current program': ['Accounting', 'Business', 'CSIS', 'Finance', 'Hospitality_Management', 'Marketing', 'Business Law', 'OADM'],
    'Year of study': ['First', 'Second', 'Third', 'Fourth'],
    'Program level': ['Bachelors', 'Certificate', 'Diploma', 'Postgraduate'],
    'Sex': ['Female', 'Male', 'Prefer not to say'],
    'Ethnicity': ['Asian', 'Hispanic/Latino', 'Native American', 'Pacific Islander', 'Prefer not to say', 'White/Caucasian'],
    'Age group': ['Under 18', '18-24', '25-34', '35-44', '45-54', '55-64', 'Prefer not to say'],
    'Used ChatGPT for learning purposes': ['Yes', 'No'],
    'How often  ChatGPT is used for educational\npurposes': ['Daily', 'Weekly', 'Monthly', 'Rarely', 'Never', 'Not Applicable'],
    'ChatGPT usage hours per week': ['Less than 1 hour', '1 to 3 hours', '3 to 5 hours', 'More than 5 hours', 'Not Applicable'],
    'Improvement of marks\nsince using ChatGPT for personalized learning': ['Yes, significant improvement', 'Yes, slight improvement', 'No change', 'Decrease in grades', 'Not Applicable'],
    'Used any features of ChatGPT that tailor\nresponses to  learning preferences or\nstyles': ['Yes', 'No', 'Not Applicable']
}

categorical_columns


{'Current program': ['Accounting',
  'Business',
  'CSIS',
  'Finance',
  'Hospitality_Management',
  'Marketing',
  'Business Law',
  'OADM'],
 'Year of study': ['First', 'Second', 'Third', 'Fourth'],
 'Program level': ['Bachelors', 'Certificate', 'Diploma', 'Postgraduate'],
 'Sex': ['Female', 'Male', 'Prefer not to say'],
 'Ethnicity': ['Asian',
  'Hispanic/Latino',
  'Native American',
  'Pacific Islander',
  'Prefer not to say',
  'White/Caucasian'],
 'Age group': ['Under 18',
  '18-24',
  '25-34',
  '35-44',
  '45-54',
  '55-64',
  'Prefer not to say'],
 'Used ChatGPT for learning purposes': ['Yes', 'No'],
 'How often  ChatGPT is used for educational\npurposes': ['Daily',
  'Weekly',
  'Monthly',
  'Rarely',
  'Never',
  'Not Applicable'],
 'ChatGPT usage hours per week': ['Less than 1 hour',
  '1 to 3 hours',
  '3 to 5 hours',
  'More than 5 hours',
  'Not Applicable'],
 'Improvement of marks\nsince using ChatGPT for personalized learning': ['Yes, significant improvement',
  'Yes

In [10]:
# Define the mappings
mappings = {
    'Satisfaction with the quality of responses by ChatGPT': {
        'Very satisfied': 5,
        'Satisfied': 4,
        'Neutral': 3,
        'Dissatisfied': 2,
        'Very dissatisfied': 1,
        'Not Applicable': 0
    },
    'How helpful ChatGPT in improving the\nunderstanding of course material': {
        'Very Helpful': 5,
        'Helpful': 4,
        'Neutral': 3,
        'Unhelpful': 2,
        'Very unhelpful': 1,
        'Not Applicable': 0
    },
    'Level of confidence in understanding of\nsubjects after using ChatGPT': {
        'Very confident': 5,
        'Confident': 4,
        'Neutral': 3,
        'Not confident': 2,
        'Not applicable': 1,
        'Not Applicable': 0
    },
    'If yes, how effective are the tailored\nresponses in enhancing comprehension and\nknowledge acquisition': {
        'Very effective': 5,
        'Effective': 4,
        'Neutral': 3,
        'Ineffective': 2,
        'Very ineffective': 1,
        'Not Applicable': 0
    },
    'Feel that personalized responses from\nChatGPT have improved grades or academic\nperformance': {
        'Strongly Agree': 5,
        'Agree': 4,
        'Neutral': 3,
        'Disagree': 2,
        'Strongly disagree': 1,
        'Not Applicable': 0
    },
    'Think that integrating ChatGPT more deeply into\nthe curriculum would benefit students': {
        'Strongly Agree': 5,
        'Agree': 4,
        'Neutral': 3,
        'Disagree': 2,
        'Strongly disagree': 1,
        'Not Applicable': 0
    }
}

# Apply the mappings
for column, mapping in mappings.items():
    df[column] = df[column].map(mapping)

print(df)


    Current program Year of study Program level                Sex  \
0              CSIS        Second  Postgraduate               Male   
1              CSIS         First       Diploma             Female   
2              CSIS        Second  Postgraduate             Female   
3          Business         Third     Bachelors               Male   
4              CSIS        Second  Postgraduate               Male   
..              ...           ...           ...                ...   
336        Business         First       Diploma  Prefer not to say   
337        Business        Second   Certificate               Male   
338        Business        Second     Bachelors             Female   
339        Business         Third       Diploma  Prefer not to say   
340        Business         Third  Postgraduate             Female   

             Ethnicity          Age group Used ChatGPT for learning purposes  \
0      Hispanic/Latino              25-34                                Yes   

In [11]:
# Function to normalize column names
def normalize_string(s):
    return s.replace(' ', '_').replace('\n', '_').replace('-', '_')

# Normalize the categorical columns' values
for column in categorical_columns.keys():
    if column in df.columns:
        df[column] = df[column].apply(normalize_string)

# Create dummy variables for each categorical feature
for column, categories in categorical_columns.items():
    if column in df.columns:
        # Create dummy variables for the current column
        dummies = pd.get_dummies(df[column], prefix=column, drop_first=False, dtype=int)
        
        # Ensure all possible categories are present by adding missing columns with zeros
        for category in categories:
            dummy_column = f"{column}_{normalize_string(category)}"
            if dummy_column not in dummies.columns:
                dummies[dummy_column] = 0
        
        # Reorder columns to ensure all categories are present
        dummies = dummies[[f"{column}_{normalize_string(category)}" for category in categories]]
        
        # Concatenate the dummy variables to the original DataFrame
        df = pd.concat([df, dummies], axis=1)
        
        # Drop the original categorical column
        df = df.drop(column, axis=1)

# Print the DataFrame to check results
print(df)

     Satisfaction with the quality of responses by ChatGPT  \
0                                                    4       
1                                                    5       
2                                                    3       
3                                                    0       
4                                                    4       
..                                                 ...       
336                                                  5       
337                                                  4       
338                                                  2       
339                                                  2       
340                                                  5       

     How helpful ChatGPT in improving the\nunderstanding of course material  \
0                                                    3                        
1                                                    5                        
2                 

In [12]:
df

,Satisfaction with the quality of responses by ChatGPT,How helpful ChatGPT in improving the\nunderstanding of course material,Level of confidence in understanding of\nsubjects after using ChatGPT,Sample situations where\nChatGPT significantly aided learning,"If yes, how effective are the tailored\nresponses in enhancing comprehension and\nknowledge acquisition",Feel that personalized responses from\nChatGPT have improved grades or academic\nperformance,Improvements to suggest for ChatGPT\nto better support learning,Think that integrating ChatGPT more deeply into\nthe curriculum would benefit students,Features that would like to be seen in ChatGPT to\nenhance personalized learning capabilities,Additional comments or suggestions regarding\nthe use of ChatGPT in learning process,...,ChatGPT usage hours per week_More_than_5_hours,ChatGPT usage hours per week_Not_Applicable,"Improvement of marks\nsince using ChatGPT for personalized learning_Yes,_significant_improvement","Improvement of marks\nsince using ChatGPT for personalized learning_Yes,_slight_improvement",Improvement of marks\nsince using ChatGPT for personalized learning_No_change,Improvement of marks\nsince using ChatGPT for personalized learning_Decrease_in_grades,Improvement of marks\nsince using ChatGPT for personalized learning_Not_Applicable,Used any features of ChatGPT that tailor\nresponses to learning preferences or\nstyles_Yes,Used any features of ChatGPT that tailor\nresponses to learning preferences or\nstyles_No,Used any features of ChatGPT that tailor\nresponses to learning preferences or\nstyles_Not_Applicable
0,4,3,3,No comment,0,3,No comment,2,No comment,No comment,...,0,0,0,0,1,0,0,0,1,0
1,5,5,5,Ask for explaining some concept,5,4,"The way ChatGPT rectify answers that I asked, ...",5,Visualization,No comment,...,0,0,1,0,0,0,0,1,0,0
2,3,4,3,Explain difficult concepts,3,3,No comment,3,More examples,Not use for exams,...,0,0,0,0,1,0,0,0,1,0
3,0,0,0,No comment,0,0,No comment,0,No comment,No comment,...,0,1,0,0,0,0,1,0,0,1
4,4,4,4,When I search for concepts that I don't unders...,5,4,more accurate amd don't fake citations,5,Summarize lectures,No comment,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,5,4,4,No comment,0,5,Enhancing the summarization capabilities for e...,5,No comment,No comment,...,0,0,1,0,0,0,0,0,1,0
337,4,4,1,Provided additional practice problems for math,2,5,Introducing personalized study plans,2,No comment,No comment,...,1,0,0,0,1,0,0,1,0,0
338,2,3,4,Assisted in learning a new programming languag...,4,3,Enhancing the feedback system for educational ...,5,No comment,No comment,...,0,0,0,0,0,1,0,1,0,0
339,2,5,1,Provided additional practice problems for math,5,2,Enhancing the summarization capabilities for e...,3,No comment,No comment,...,0,0,0,0,0,1,0,1,0,0


In [13]:
df.columns

Index(['Satisfaction with the quality of responses by ChatGPT',
       'How helpful ChatGPT in improving the\nunderstanding of course material',
       'Level of confidence in understanding of\nsubjects after using ChatGPT',
       'Sample situations where\nChatGPT significantly aided learning',
       'If yes, how effective are the tailored\nresponses in enhancing comprehension and\nknowledge acquisition',
       'Feel that personalized responses from\nChatGPT have improved grades or academic\nperformance',
       'Improvements to suggest for ChatGPT\nto better support learning',
       'Think that integrating ChatGPT more deeply into\nthe curriculum would benefit students',
       'Features that would like to be seen in ChatGPT to\nenhance  personalized learning capabilities',
       'Additional comments or suggestions regarding\nthe use of ChatGPT in learning process',
       'Auditory', 'Kinesthetic', 'Reading/Writing', 'Usage Patterns',
       'Visual', 'Assignments', 'Class lectu

In [14]:
from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(df['Level of confidence in understanding of\nsubjects after using ChatGPT'], df['Satisfaction with the quality of responses by ChatGPT'])

# Perform Chi-Square test
chi2, p, dof, ex = chi2_contingency(contingency_table, correction=False)
print(f"Chi-Square Statistic: {chi2}, P-value: {p}")


Chi-Square Statistic: 469.32162537480116, P-value: 1.7126555874359135e-83


In [15]:
pip install statsmodels


Note: you may need to restart the kernel to use updated packages.


In [16]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.miscmodels.ordinal_model import OrderedModel

# Define the ordinal logistic regression model
model = OrderedModel(
    endog=df['If yes, how effective are the tailored\nresponses in enhancing comprehension and\nknowledge acquisition'],  # Dependent variable 
    exog=df[['How helpful ChatGPT in improving the\nunderstanding of course material', 'Level of confidence in understanding of\nsubjects after using ChatGPT', 'Satisfaction with the quality of responses by ChatGPT', 'Feel that personalized responses from\nChatGPT have improved grades or academic\nperformance', 'Think that integrating ChatGPT more deeply into\nthe curriculum would benefit students']],  # Independent variables
    distr='logit'  # Using the logit link function
)

# Fit the model
result = model.fit(method='bfgs')

# Print the summary
print(result.summary())

Optimization terminated successfully.
         Current function value: 1.008976
         Iterations: 49
         Function evaluations: 51
         Gradient evaluations: 51
                                                                       OrderedModel Results                                                                      
Dep. Variable:     If yes, how effective are the tailored
responses in enhancing comprehension and
knowledge acquisition   Log-Likelihood:                -344.06
Model:                                                                                                      OrderedModel   AIC:                             708.1
Method:                                                                                               Maximum Likelihood   BIC:                             746.4
Date:                                                                                                   Thu, 22 Aug 2024                                         
Time:             